In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from plaid.denoisers import UIPA
ckpt_path = "/shared/amyxlu/dprot/ckpts/2eiqqk2u"
denoiser = UIPA.load_from_checkpoint(ckpt_path)

In [4]:
from plaid.diffusion import GaussianDiffusion
diffusion = GaussianDiffusion(
    model=denoiser,
    sampling_timesteps=100,
)

In [18]:
import torch
batch_size = 16
UNCOND_IDX = 6
x = torch.full((batch_size, 3),  UNCOND_IDX).long()
cond_dict = {"secondary_structure": x}
model_kwargs = {"cond_dict": cond_dict}

In [20]:
sampled = diffusion.sample(
    shape=(16,64,1024),
    return_all_timesteps=True,
    model_kwargs=model_kwargs
)

sampling loop time step:   0%|          | 0/100 [00:00<?, ?it/s]

In [22]:
print(sampled.shape)
x = sampled[:, -1, :, :]
print(x.shape)

torch.Size([16, 101, 64, 1024])
torch.Size([16, 64, 1024])


In [23]:
from plaid.callbacks import SampleCallback

sampler = SampleCallback(
    diffusion=diffusion,
    model=denoiser,
    calc_fid=True
)

In [25]:
device = torch.device("cuda")

In [27]:
strs, log_dict = sampler.construct_sequence(x, device)

percentage similarty to argmax idx: 0.987
Perplexity: 15.695


In [29]:
device = torch.device("cuda:2")

In [31]:
pdb_strs, metrics, log_dict = sampler.construct_structure(x, strs, device)

(Generating structure from latents..):   0%|          | 0/1 [00:00<?, ?it/s]

(Generating structure from latents..): 100%|██████████| 1/1 [07:08<00:00, 428.54s/it]


In [39]:
from plaid.utils import view_py3Dmol
view_py3Dmol(pdb_strs[0])

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol